In [2]:
#Importing the needed packages
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline 

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb

In [3]:
#loading the data
expedia_df = pd.read_csv('training_set_VU_DM_2014.csv')
test_df = pd.read_csv('test_set_VU_DM_2014.csv')

In [9]:
#more usefull date information training set by splitting into year and month
expedia_df["date_time"] = pd.to_datetime(expedia_df["date_time"])
expedia_df["year"] = expedia_df["date_time"].dt.year
expedia_df["month"] = expedia_df["date_time"].dt.month

In [10]:
#more usefull date information test set by splitting into year and month
test_df["date_time"] = pd.to_datetime(test_df["date_time"])
test_df["year"] = test_df["date_time"].dt.year
test_df["month"] = test_df["date_time"].dt.month
